In [1]:
import pandas as pd
from bs4 import BeautifulSoup, NavigableString
import requests
import time
import pickle
import os
import json
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
class DFStorage:
    def __init__(self):
        self.df: pd.DataFrame = pd.DataFrame(columns=['name', 'country', 'city', 'link', 'category', 'pub_language', 'n_release_in_year', 'multidisciplinary', ])
    
    def append(self, data: dict):
        self.df = pd.concat([self.df, pd.DataFrame(data, index=[0])], ignore_index=True)

    def save_to_csv(self, postfix: str):
        self.df.to_csv(f'./part_{postfix}.csv')
        self.df = pd.DataFrame()
    
    def __str__(self):
        return str(len(self.df))

In [9]:
requests.get('https://elibrary.ru')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))

In [ ]:
class Parser:
    def __init__(self, max_magazines=100, storage: DFStorage=None):
        self.max_magazines = max_magazines
        self.magazine_links = []
        self.base_url = 'https://elibrary.ru'
        # self.storage = storage
        self.result = {}
        self.cookies = None

    def _get_page_source(self, url):
        options = Options()
        # options.add_argument('--headless')
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        for cookie in self.cookies:
            driver.add_cookie(cookie_dict=cookie)
        driver.get(url)

        try:
            print('capcha')
            WebDriverWait(driver, 300).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'header-panel'))
            )
            print('passed')
        except:
            raise Exception('capcha error')
            
        page_source = driver.page_source
        driver.quit()
        return page_source
    
    def _auth(self):
        options = Options()
        # options.add_argument('--headless')
        driver = webdriver.Firefox(options=options)
        driver.get(self.base_url)

        try:
            print('auth')
            WebDriverWait(driver, 600).until(
                EC.invisibility_of_element_located((By.ID, 'win_login'))
            )
            print('passed')
        except:
            raise Exception('auth error')
            
        self.cookies = driver.get_cookies()
        driver.quit()
    
    def _parse_categories(self):
        page_source = self._get_page_source(url=self.base_url + '/titles.asp')
        soup = BeautifulSoup(page_source, 'html.parser')
        select = soup.find('select', attrs={'id': 'rubriccode'})
        options = select.find_all('option')
        categories = {}
        for option in options:
            if option.text.strip() == '':
                continue
            option_name = option.text.strip().replace('\xa0', '/').split('/')[0]
            categories[option_name] = option.get('value')
        with open('categories.json', 'w') as file:
            json.dump(categories, file, ensure_ascii=False, indent=4)
        return categories
    
    def _parse_magazine_links(self, page: str):
        soup = BeautifulSoup(page, 'html.parser')
        list_of_magazines = soup.find('table', attrs={'id': 'restab'})
        list_items = list_of_magazines.find_all('td', attrs={'align': 'left', 'class': 'midtext'})
        links = [self.base_url + '/' + td.find('a').get('href') for td in list_items]
        return links

    def _parse_link_page(self, params):
        try:
            params_str = '?' + '&'.join([f'{param}={params[param]}' for param in params])
            page_source = page_source = self._get_page_source(url=self.base_url + '/titles.asp' + params_str)
            links = self._parse_magazine_links(page_source)
            return links
        except Exception as e:
            with open('error.txt', 'a') as file:
                file.write(f'{page_source} {params} {e}')
            raise e
    
    def _parse_link_pages(self):
        if 'magazine_links.json' in os.listdir('./'):
            with open('magazine_links.json', 'r') as file:
                magazine_links = json.load(file)
            return magazine_links
        
        categories = self._parse_categories()
        print(categories)
        magazine_links = {}
        for category in categories:
            category_magazine_links = []
            page = 1
            while len(category_magazine_links) < self.max_magazines:
                params = {'pagenum': page, 'rubriccode': categories[category], 'sortorder': 4, 'order': 1}
                try:
                    links = self._parse_link_page(params=params)
                except Exception:
                    continue
                print(len(links), links[:3])
                category_magazine_links.extend(links[: self.max_magazines] if len(links) > self.max_magazines else links)
                print(params)
                if len(links) < 100: 
                    break
                page += 1
            magazine_links[category] = category_magazine_links

        with open('magazine_links.json', 'w') as file:
            json.dump(magazine_links, file)

        return magazine_links
            
    def _parse_magazine_page(self, soup: BeautifulSoup):
        raw = {}
        
        for img in soup.find_all("img", class_="imghelp"):
            parent = img.parent
            label = None
            for sib in parent.contents:
                if sib is img:
                    continue
                if isinstance(sib, NavigableString):
                    text = sib.strip()
                    if text and ':' in text:
                        label = text.split(':', 1)[0].strip()
                        break
            if not label:
                continue

            val = None
            font = parent.find("font", attrs={"color": "#00008f"})
            if font and font.get_text(strip=True):
                val = font.get_text(strip=True)
            else:
                a = parent.find("a", href=True)
                if a and a.get_text(strip=True):
                    val = a.get_text(strip=True)

            raw[label] = val
        return raw

    def parse(self):
        self._auth()
        magazine_links = self._parse_link_pages()
        res = {}

        try:
            for category in magazine_links:
                for link in magazine_links[category]:
                    page_source = self._get_page_source(link)
                    soup = BeautifulSoup(page_source)
                    soup = soup.select('#thepage > table > tbody > tr > td > table:nth-child(1) > tbody > tr > td:nth-child(2) > form > table > tbody > tr:nth-child(2) > td:nth-child(1) > table > tbody > tr > td')[0]
                    raw = self._parse_magazine_page(soup)
                    data = {
                        'Название журнала': soup.find("span", class_="bigtext").get_text(strip=True),
                        'Страна': raw.get('Страна'),
                        'Город': raw.get('Город'),
                        'Сайт': raw.get('Сайт'),
                        'Число выпусков в год': raw.get('Число выпусков в год'),
                        'Мультидисциплинарность': raw.get('Мультидисциплинарность'),
                        'Тип сериального издания': raw.get('Тип сериального издания'),
                        'Элементы сериального издания': raw.get('Элементы сериального издания'),
                        'Назначение издания': raw.get('Назначение издания'),
                        'Число статей в выпуске': raw.get('Число статей в выпуске')
                    }
                    print(data)
                    if category not in res:
                        res[category] = []
                    res[category].append(data)
        except Exception:
            print(link)

        with open('data.json', 'w') as file:
            json.dump(res, file)
        # self.storage.save_to_csv(f'{self.start_page}_{self.end_page}')


In [13]:
# storage = DFStorage()
parser = Parser(max_magazines=20)
target=parser.parse()

auth
passed
capcha
passed
{'Автоматика. Вычислительная техника': '500000', 'Астрономия': '410000', 'Биология': '340000', 'Биотехнология': '620000', 'Внешняя торговля': '720000', 'Внутренняя торговля. Туристско-экскурсионное обслуживание': '710000', 'Водное хозяйство': '700000', 'Военное дело': '780000', 'География': '390000', 'Геодезия. Картография': '360000', 'Геология': '380000', 'Геофизика': '370000', 'Горное дело': '520000', 'Государство и право. Юридические науки': '100000', 'Демография': '050000', 'Жилищно-коммунальное хозяйство. Домоводство. Бытовое обслуживание': '750000', 'Информатика': '200000', 'Искусство. Искусствоведение': '180000', 'История. Исторические науки': '030000', 'Кибернетика': '280000', 'Комплексное изучение отдельных стран и регионов': '230000', 'Комплексные проблемы общественных наук': '260000', 'Космические исследования': '890000', 'Культура. Культурология': '130000', 'Легкая промышленность': '640000', 'Лесная и деревообрабатывающая промышленность': '660000',

KeyboardInterrupt: 

In [ ]:
 try:
    while True:
        pass
except KeyboardInterrupt:
    print(link)

KeyboardInterrupt: 